# Web scrapping for Toronto wikipedia to create intial dataset of Toronto

This notebook is intended for Capstone project week 3. 
In this assignment, we will collect data of Toronto city which has Postal Code, Borough, Neighborhood . We will pulling out the data from Toronto's wikipedia ( https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M ) using BeautifulSoup to read and parse Toronto's wikipedia and save the list in csv file (Toronto.csv) .  

In [0]:
# function to return key for any value 
def get_key(val, my_dict): 
    for key, value in my_dict.items(): 
      if key == val: 
             return key 
  
    return "key doesn't exist"

In [0]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'html5lib')

# create an empty dataframe
columns = ['PostalCode', 'Borough', 'Neighborhood']
toronto_data = pd.DataFrame(columns=columns)
toronto_data

table_list = soup.find_all("table")[0]
row_list = table_list.find_all("tr")

postal_codes_dict = {} # initialize an empty dictionary to save the data in

for row in row_list:
  td_list = row.find_all('td') 

  #if it header, skip
  if (len(td_list) > 0):
    postal_code = td_list[0].text
    borough = td_list[1].text
    neighborhood = td_list[2].text.replace('\n','')

    #if the borough is not assign, skip
    if (borough != 'Not assigned'):

      #if the neighborhood is not assign, copy from borough
      if (neighborhood == 'Not assigned'):
        neighborhood = borough

      #if the postal code hasn't been found, create a new one
      if (get_key(postal_code, postal_codes_dict) == "key doesn't exist"):
        postal_codes_dict[postal_code] = {} 
        postal_codes_dict[postal_code]['borough'] = borough
        postal_codes_dict[postal_code]['neighborhoods'] = neighborhood
      #if the postal code has been found, concat with the old one
      else:
        new_neighborhoods = postal_codes_dict[postal_code]['neighborhoods'] + ', ' + neighborhood
        postal_codes_dict[postal_code]['neighborhoods'] = new_neighborhoods

# populate dataframe with data from dictionary
for ind, postal_code in enumerate(postal_codes_dict):
    borough = postal_codes_dict[postal_code]['borough']
    neighborhood = postal_codes_dict[postal_code]['neighborhoods']
    toronto_data = toronto_data.append({"PostalCode": postal_code, 
                                        "Borough": borough, 
                                        "Neighborhood": neighborhood},
                                        ignore_index=True)

# print number of rows of dataframe
toronto_data.shape[0]

103

In [0]:
toronto_data

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
101,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."


In [54]:
#save toronto_data to Toronto.csv
from google.colab import drive

drive.mount('drive')
toronto_data.to_csv('Toronto.csv')
!cp Toronto.csv "drive/My Drive/Colab Notebooks"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive
